In [1]:
#nltk
from spacy import displacy
import spacy
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#Adapted from Abid Ali Awan
#https://deepnote.com/@abid/spaCy-Resume-Analysis-81ba1e4b-7fa8-45fe-ac7a-0b7bf3da7826
#warning
import warnings 
warnings.filterwarnings('ignore')

class SkillsExtractor:
    def __init__(self, skill_pattern_path):
        self.skill_pattern_path = skill_pattern_path
        self.nlp = ""
        self.colors = {
        "SKILL": "linear-gradient(90deg, #9BE15D, #00E3AE)",
        "SOFT-SKILL": "linear-gradient(20deg, #FFAFBD, #FFC3A0)"
        }
        self.options = {
            "ents": [
                "SKILL",
                "SOFT-SKILL" #ORIGINAL ADDITION 
            ],
            "colors": self.colors
        }

    def setNLP(self, nlp):#ORIGINAL ADDITION 
        self.nlp = nlp #ORIGINAL ADDITION 

    def getNLP(self):#ORIGINAL ADDITION 
        return self.nlp #ORIGINAL ADDITION 
        
    def start(self, nlp=None):
        if nlp == None:
            nlp = spacy.load("en_core_web_sm")
        else:#ORIGINAL ADDITION 
            self.setNLP(nlp)
        ruler = self.getNLP().add_pipe("entity_ruler") 
        ruler.from_disk(self.skill_pattern_path) 
        self.setNLP(nlp)#ORIGINAL ADDITION 
        
    #Getters from spacy resume tool not my original code!
    def get_skills(self, text):
        nlp = self.getNLP()#ORIGINAL ADDITION 
        doc = nlp(text)
        myset = []
        subset = []
        for ent in doc.ents:
            if ent.label_ == "SKILL":
                subset.append(ent.text)
        myset.append(subset)
        return subset

    def get_soft_skills(self, text):
        nlp = self.getNLP()#ORIGINAL ADDITION 
        doc = nlp(text)
        myset = []
        subset = []
        for ent in doc.ents:
            if ent.label_ == "SOFT-SKILL":#ORIGINAL ADDITION 
                subset.append(ent.text)#ORIGINAL ADDITION 
        myset.append(subset)
        return subset

    def unique_skills(self, x):
        return list(set(x))

    def display(self, text): 
        nlp = self.getNLP()#ORIGINAL ADDITION 
        docx = nlp(text)
        html = displacy.render(docx, style='ent', options=self.options)
        return html

# Copy model-last from the Flask Demo

In [228]:
skill_pattern_path = "data/jz_skill_patternsV3.jsonl"
SE = SkillsExtractor(skill_pattern_path)
nlp = spacy.load("model-last")
#nlp = spacy.load("en_core_web_sm")
SE.start(nlp)

# Import train or test annoted data
Swap between train annoted data 
or testing annoted data

In [220]:
import json
#f = open('data/train.jsonl')
f = open('data/test.jsonl')
array = []
for entity in f:
   array.append( (json.loads(entity)['data'] , json.loads(entity)['label']) )

In [221]:
testData = []
for data, label in array:
    skills = []
    for e in label:
        skills.append(data[e[0]:e[1]])
    testData.append(skills)

In [242]:
index = (int)(input()) # up to 167 for train data, and 35 for test data

34


In [243]:
resume = array[index][0]
skills = SE.unique_skills(SE.get_skills(resume.lower()))
softSkills = SE.unique_skills(SE.get_soft_skills(resume.lower()))
print(softSkills)

['accountability', 'lead', 'manages multiple tasks', 'written and verbal communication', 'organized', 'problem solving', 'management', 'coordinated', 'timely', 'work well within a team', 'organization', 'interacts easily with people of diverse backgrounds', 'organizational and time management skills', 'organizational', 'customer service', 'public relations skills', 'interpersonal relations', 'communication', 'solid']


In [244]:
def score(PredictedSkills, Actualskills):
    score = 0
    length = len(Actualskills)
    matched = []
    for skill in PredictedSkills:
        if skill in Actualskills:
            matched.append(skill)
            score += 1
    skillsNotMet = list(set(Actualskills) - set(matched)) 
    return (score, length, skillsNotMet)

In [245]:
score(softSkills, testData[index])

(9,
 12,
 ['strong interpersonal relations', 'written and verbal communication skills'])

In [229]:
i = 0
totalscore = 0
totalLen = 0
totalskills = []
skillsToAdd = []
for entity in array:
    resume = entity[0]
    for e in entity[1]:
        totalskills.append(e)
    softSkills = SE.unique_skills(SE.get_soft_skills(resume.lower()))
    scoreTuple = score(softSkills, testData[i])
    print(scoreTuple)
    totalscore = totalscore + scoreTuple[0]
    totalLen =  totalLen + scoreTuple[1]
    #print(totalscore)
    #print(totalLen)
    for e in scoreTuple[2]:
        skillsToAdd.append(e)
    i += 1
matched = (totalscore/totalLen)*100
print("score: {:.3f}%".format(matched))

(4, 4, [])
(2, 2, [])
(7, 7, [])
(7, 8, ['good working relations'])
(2, 2, [])
(1, 1, [])
(3, 3, [])
(1, 1, [])
(5, 6, ['written and spoken english'])
(3, 4, ['active listening'])
(4, 4, [])
(0, 0, [])
(14, 17, ['ability to articulate your ideas', 'research data'])
(0, 0, [])
(4, 5, ['communicate'])
(5, 7, ['problem solving', 'customer service skills'])
(3, 3, [])
(0, 0, [])
(3, 4, ['fluent in english'])
(2, 2, [])
(3, 3, [])
(13, 14, [])
(17, 23, ['strong leader'])
(11, 13, ['written oral and interpersonal communication skills', 'developed solid relationships'])
(8, 10, ['collaborating', 'active listening'])
(7, 7, [])
(11, 13, [])
(7, 13, ['written and oral', 'technical orientation', 'communicator'])
(2, 2, [])
(4, 4, [])
(6, 9, [])
(6, 7, ['leadership'])
(6, 8, ['quality driven'])
(2, 2, [])
(9, 12, ['strong interpersonal relations', 'written and verbal communication skills'])
score: 82.727%


In [ ]:
import pandas as pd
df = pd.DataFrame(skillsToAdd, columns = ['soft_skill'])
#df.to_csv('data/trainSoftSkills.csv')
df.to_csv('data/testSoftSkills.csv')

In [71]:
resume = input()
SE.display(resume)

In [173]:
skills = SE.get_skills(resume)
print(skills)
skills = SE.get_soft_skills(resume)
print(skills)

['software', 'HTML5', 'version control system', 'React', 'Ember', 'Angular', 'server', 'languages', 'PHP', 'Python', 'framework', 'React', 'databases', 'CSS', 'SASS', 'CSS']
['accurately inform']


In [176]:
docx = nlp(resume)

In [180]:
colors = {
        "SKILL": "linear-gradient(90deg, #9BE15D, #00E3AE)",
        "SOFT-SKILL": "linear-gradient(20deg, #FFAFBD, #FFC3A0)"
        }
options = {
            "ents": [
                "SKILL",
                "SOFT-SKILL"
            ],
            "colors": colors
        }

In [181]:
html = displacy.render(docx, style='ent', options=options)
html